In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
import os
sys.path.append("..")

In [3]:
from utils import load_citation, sgc_precompute
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
import pickle as pkl
from train import train_gfnn

# Cora

In [43]:
adj, features, labels, idx_train,\
idx_val, idx_test = load_citation("cora",
                                  "LeftNorm",
                                  False,
                                  0.6,
                                  False)

In [44]:
feature, _ = sgc_precompute(features, adj, 2)

In [45]:
from sklearn.ensemble import RandomForestClassifier

In [46]:
X = feature.numpy()
y = labels.numpy()
idx_train = idx_train.numpy()
idx_val = idx_val.numpy()
idx_test = idx_test.numpy()

In [47]:
clf = RandomForestClassifier(n_estimators=100)

In [48]:
clf.fit(X[idx_train], y[idx_train])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [49]:
clf.score(X[idx_test], y[idx_test])

0.781

In [50]:
from sklearn.svm import SVC

In [51]:
clf = SVC(C=1000.0, gamma=0.2, class_weight="balanced")

In [52]:
clf.fit(X[idx_train], y[idx_train])

SVC(C=1000.0, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.2, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [53]:
clf.score(X[idx_test], y[idx_test])

0.792

In [54]:
for C in [1.0, 10.0, 100.0, 1000.0]:
    for g in [0.01, 0.1, 0.2, 0.5, 1.0]:
        clf = SVC(C=C, gamma=g, class_weight="balanced")
        clf.fit(X[idx_train], y[idx_train])
        print(C, g, clf.score(X[idx_test], y[idx_test]))

1.0 0.01 0.433
1.0 0.1 0.433
1.0 0.2 0.433
1.0 0.5 0.437
1.0 1.0 0.445
10.0 0.01 0.433
10.0 0.1 0.433
10.0 0.2 0.433
10.0 0.5 0.474
10.0 1.0 0.759
100.0 0.01 0.433
100.0 0.1 0.766
100.0 0.2 0.788
100.0 0.5 0.798
100.0 1.0 0.795
1000.0 0.01 0.768
1000.0 0.1 0.793
1000.0 0.2 0.792
1000.0 0.5 0.789
1000.0 1.0 0.788


In [103]:
for C in [20, 30, 50, 100.0, 150, 200]:
    for g in [0.01, 0.1, 0.2, 0.5, 1.0]:
        clf = SVC(C=C, gamma=g, class_weight="balanced")
        clf.fit(X[idx_train], y[idx_train])
        print(C, g, clf.score(X[idx_test], y[idx_test]))

20 0.01 0.431
20 0.1 0.431
20 0.2 0.431
20 0.5 0.755
20 1.0 0.783
30 0.01 0.431
30 0.1 0.431
30 0.2 0.537
30 0.5 0.774
30 1.0 0.795
50 0.01 0.431
50 0.1 0.448
50 0.2 0.754
50 0.5 0.794
50 1.0 0.798
100.0 0.01 0.431
100.0 0.1 0.755
100.0 0.2 0.784
100.0 0.5 0.798
100.0 1.0 0.801
150 0.01 0.431
150 0.1 0.777
150 0.2 0.798
150 0.5 0.798
150 1.0 0.8
200 0.01 0.431
200 0.1 0.785
200 0.2 0.802
200 0.5 0.8
200 1.0 0.796


In [104]:
for C in [100.0, 110, 120]:
    for g in [1.0, 1.2, 1.5, 2.0, 4.0]:
        clf = SVC(C=C, gamma=g, class_weight="balanced")
        clf.fit(X[idx_train], y[idx_train])
        print(C, g, clf.score(X[idx_test], y[idx_test]))

100.0 1.0 0.801
100.0 1.2 0.804
100.0 1.5 0.8
100.0 2.0 0.792
100.0 4.0 0.794
110 1.0 0.804
110 1.2 0.802
110 1.5 0.8
110 2.0 0.795
110 4.0 0.793
120 1.0 0.804
120 1.2 0.8
120 1.5 0.799
120 2.0 0.791
120 4.0 0.795


In [55]:
from hpsklearn import HyperoptEstimator, svc
from sklearn import svm

In [56]:
estim = HyperoptEstimator(classifier=svc('mySVC'))

In [59]:
estim.fit(X[np.concatenate((idx_train, idx_val))], y[np.concatenate((idx_train, idx_val))])

100%|██████████| 10/10 [00:00<00:00, 11.66trial/s, best loss: 0.15625]


In [60]:
print(estim.score(X[idx_test], y[idx_test]))

0.829


# Citeseer

In [19]:
adj, features, labels, idx_train,\
idx_val, idx_test = load_citation("citeseer",
                                  "LeftNorm",
                                  False,
                                  0.6,
                                  False)
feature, _ = sgc_precompute(features, adj, 2)

X = feature.numpy()
y = labels.numpy()
idx_train = idx_train.numpy()
idx_val = idx_val.numpy()
idx_test = idx_test.numpy()

../normalization.py:73: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1).flatten()


In [20]:
estim = HyperoptEstimator(classifier=svc('mySVC'))

In [21]:
estim.fit(X[np.concatenate((idx_train, idx_val))], y[np.concatenate((idx_train, idx_val))])

100%|██████████| 10/10 [00:01<00:00,  5.84trial/s, best loss: 0.17741935483870963]


In [22]:
print(estim.score(X[idx_test], y[idx_test]))

0.723


# Pubmed

In [23]:
adj, features, labels, idx_train,\
idx_val, idx_test = load_citation("pubmed",
                                  "LeftNorm",
                                  False,
                                  0.6,
                                  False)
feature, _ = sgc_precompute(features, adj, 2)

X = feature.numpy()
y = labels.numpy()
idx_train = idx_train.numpy()
idx_val = idx_val.numpy()
idx_test = idx_test.numpy()

In [24]:
estim = HyperoptEstimator(classifier=svc('mySVC'))

In [27]:
estim.fit(X[np.concatenate((idx_train, idx_val))], y[np.concatenate((idx_train, idx_val))])

100%|██████████| 10/10 [00:00<00:00, 97.08trial/s, best loss: 0.1875]


In [28]:
print(estim.score(X[idx_test], y[idx_test]))

0.815


In [30]:
estim.best_model()

{'learner': SVC(C=280.7917446209502, cache_size=512, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=1, gamma='auto', kernel='linear',
     max_iter=175126193.0, probability=False, random_state=0, shrinking=False,
     tol=0.001269905373417759, verbose=False),
 'preprocs': (),
 'ex_preprocs': ()}

In [37]:
clf = SVC(C=280.7917446209502, cache_size=512, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=1, gamma='auto', kernel='linear',
     max_iter=175126193.0, probability=False, random_state=0, shrinking=False,
     tol=0.001269905373417759, verbose=False)

In [41]:
clf.fit(X[np.concatenate((idx_train,idx_val))], y[np.concatenate((idx_train,idx_val))])

SVC(C=280.7917446209502, cache_size=512, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=1, gamma='auto', kernel='linear',
    max_iter=175126193.0, probability=False, random_state=0, shrinking=False,
    tol=0.001269905373417759, verbose=False)

In [42]:
print(clf.score(X[idx_test], y[idx_test]))

0.815
